In [1]:
import pandas as pd
from collections import Counter
from numba import njit
import numpy as np
# !pip install line_profiler
# %load_ext line_profiler

In [2]:
recipes = pd.read_csv('recipes_sample.csv', sep=',', parse_dates=['submitted'])
recipes

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients
0,george s at the cove black bean soup,44123,90,35193,2002-10-25,NaN,an original recipe created by chef scott meska...,18.0
1,healthy for them yogurt popsicles,67664,10,91970,2003-07-26,NaN,my children and their friends ask for my homem...,NaN
2,i can t believe it s spinach,38798,30,1533,2002-08-29,NaN,"these were so go, it surprised even me.",8.0
3,italian gut busters,35173,45,22724,2002-07-27,NaN,my sister-in-law made these for us at a family...,NaN
4,love is in the air beef fondue sauces,84797,25,4470,2004-02-23,4.0,i think a fondue is a very romantic casual din...,NaN
...,...,...,...,...,...,...,...,...
29995,zurie s holey rustic olive and cheddar bread,267661,80,200862,2007-11-25,16.0,this is based on a french recipe but i changed...,10.0
29996,zwetschgenkuchen bavarian plum cake,386977,240,177443,2009-08-24,NaN,"this is a traditional fresh plum cake, thought...",11.0
29997,zwiebelkuchen southwest german onion cake,103312,75,161745,2004-11-03,NaN,this is a traditional late summer early fall s...,NaN
29998,zydeco soup,486161,60,227978,2012-08-29,NaN,this is a delicious soup that i originally fou...,NaN


In [3]:
reviews = pd.read_csv('reviews_sample.csv', sep=',', index_col=0, parse_dates=['date'])
reviews

FileNotFoundError: [Errno 2] No such file or directory: 'reviews_sample.csv'

Реализуйте несколько вариантов функции подсчета среднего значения столбца rating из таблицы reviews для отзывов, оставленных в 2010 году

In [52]:
def rating_mean_first(reviews: pd.DataFrame):
    rating_sum_, rows_sum_ = 0, 0
    for index, row in reviews.iterrows():
        if row.date.year == 2010:
            rating_sum_ += row.rating
            rows_sum_ += 1
    return rating_sum_ / rows_sum_

%timeit rating_mean_first(reviews)
f'result: {rating_mean_first(reviews)}'

7.51 s ± 246 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


'result: 4.4544402182900615'

In [53]:
def rating_mean_second(reviews: pd.DataFrame):
    filtered_reviews = reviews[reviews.date.dt.year == 2010]
    rating_sum_, rows_sum_ = 0, 0
    for index, row in filtered_reviews.iterrows():
        rating_sum_ += row.rating
        rows_sum_ += 1
    return rating_sum_ / rows_sum_

%timeit rating_mean_second(reviews)
f'result: {rating_mean_second(reviews)}'

681 ms ± 8.25 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


'result: 4.4544402182900615'

In [54]:
def rating_mean_third(reviews: pd.DataFrame):
    return reviews[reviews.date.dt.year == 2010].rating.mean()

%timeit rating_mean_second(reviews)
f'result: {rating_mean_second(reviews)}'

691 ms ± 22.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


'result: 4.4544402182900615'

In [7]:
assert rating_mean_first(reviews) == rating_mean_second(reviews) == rating_mean_third(reviews)

2.Какая из созданных функций выполняется медленнее? Что наиболее сильно влияет на скорость выполнения? Для ответа использовать профайлер line_profiler. Сохраните результаты работы профайлера в отдельную текстовую ячейку и прокомментируйте результаты его работы.

In [55]:
%lprun -f rating_mean_first rating_mean_first(reviews)

(*). Сможете ли вы ускорить работу функции 1Б, отказавшись от использования метода iterrows, но не используя метод mean?

In [19]:
def rating_mean_second(reviews: pd.DataFrame):
    filtered_reviews = reviews[reviews.date.dt.year == 2010]
    return filtered_reviews.rating.sum() / filtered_reviews.shape[0]

%timeit rating_mean_second(reviews)
f'result: {rating_mean_second(reviews)}'

7.38 ms ± 634 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


'result: 4.4544402182900615'

3.Вам предлагается воспользоваться функцией, которая собирает статистику о том, сколько отзывов содержат то или иное слово. Измерьте время выполнения этой функции. Сможете ли вы найти узкие места в коде, используя профайлер? Выпишите (словами), что в имеющемся коде реализовано неоптимально. Оптимизируйте функцию и добейтесь значительного (как минимум, на один порядок) прироста в скорости выполнения.

In [103]:
def get_word_reviews_count(df):
    words_count = Counter()
    review = df.dropna(subset=['review'])['review'].str.lower().str.split(' ').explode()
    for word in review:
        words_count[word] += 1
#     Counter(review)
    return words_count

In [104]:
%timeit get_word_reviews_count(reviews)
# get_word_reviews_count(reviews)

3.55 s ± 401 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


4.Напишите несколько версий функции MAPE (см. MAPE) для расчета среднего абсолютного процентного отклонения значения рейтинга отзыва на рецепт от среднего значения рейтинга по всем отзывам для этого рецепта.

A. Без использования векторизованных операций и методов массивов numpy и без использования numba
B. Без использования векторизованных операций и методов массивов numpy, но с использованием numba
C. С использованием векторизованных операций и методов массивов numpy, но без использования numba
D. C использованием векторизованных операций и методов массивов numpy и numba

In [4]:
reviews = reviews[['recipe_id', 'rating']].dropna()
reviews = reviews[reviews['rating'] != 0]
grouped_by_recipe_id = reviews.groupby(['recipe_id'])['rating']

In [11]:
def mape_one(data):
    mean = data.mean()
    return sum([abs(i - mean) / i for i in data]) / len(data) * 100

%timeit grouped_by_recipe_id.apply(mape_one)

1.5 s ± 66.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [41]:
@njit
def mape_1(data, mean):
    return sum([abs(i - mean) / i for i in data]) / len(data) * 100

def a(values, index):
    return mape_1(values, values.mean())

%timeit grouped_by_recipe_id.agg(a, engine='numba')

542 ms ± 21.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [39]:
def mape_one(data):
    mean = data.mean()
    return np.mean(np.fabs((data - mean) / data) * 100

%timeit grouped_by_recipe_id.apply(mape_one)

8.93 s ± 313 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [42]:
@njit
def mape_1(data, mean):
    return np.mean(np.fabs(data - mean / data)) * 100

def a(values, index):
    return mape_1(values, values.mean())

%timeit grouped_by_recipe_id.agg(a, engine='numba')

506 ms ± 9.45 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
